<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax Scaler Informer](#3-minmax-scaler-informer)
- [4. MinMax Scaler PatchTST](#4-minmax-scaler-patchtst)

</details>


Here we perform a check on **Germany** dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

**For Standard Scaler and MinMax we tried learning rates: 0.0001, 0.00001, 0.000001.**

In [1]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [17]:
cuda_device = "1"

# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Set the best learning rate based on pred_len
            if pred_len == "24":
                lr = 0.00001
            elif pred_len in ["96", "168"]:
                lr = 0.0001
            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --dropout 0.1 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MAE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', if_relu=False, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=True, loss_fnc='MAE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience

In [36]:
path_dir = './dataset_results'

if not os.path.exists(path_dir):
    os.makedirs(path_dir)

csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.4982  0.7059  0.4920  0.5586
              2         24        0.4970  0.7050  0.4928  0.5580
              1         96        0.8278  0.9098  0.6768  0.7216
              2         96        0.8712  0.9334  0.6843  0.7403
              1         168       0.9250  0.9618  0.7023  0.7619
              2         168       0.8726  0.9341  0.6942  0.7400
MAE           1         24        0.4726  0.6874  0.4501  0.5441
              2         24        0.4858  0.6970  0.4535  0.5516
              1         96        0.9520  0.9757  0.6667  0.7739
              2         96        0.8675  0.9314  0.6356  0.7387
              1         168       0.9182  0.9582  0.6792  0.7591
              2         168       0.9472  0.9732  0.6660  0.7710

In [37]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        20647858.0  4543.9912  2991.8401  0.2259
              2         24        20012764.0  4473.5630  2979.8721  0.2224
              1         96        35706256.0  5975.4712  4163.1543  0.2976
              2         96        38053568.0  6168.7573  4203.2881  0.3072
              1         168       41039112.0  6406.1777  4328.8550  0.3192
              2         168       37693112.0  6139.4717  4274.0054  0.3059
MAE           1         24        18663196.0  4320.0923  2689.3916  0.2148
              2         24        19648218.0  4432.6309  2722.6453  0.2204
              1         96        41247308.0  6422.4067  4063.6636  0.3198
              2         96        37667352.0  6137.3735  3862.1619  0.3056
              1         168       39692484.0  6300.1973  4161.8623  0.3139
              2         168       40591140.0  6371.1177  4053.1790  0.3174

In [38]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.4792  0.6922  0.4518  0.5478
         MSE            0.4976  0.7054  0.4924  0.5583
96       MAE            0.9098  0.9536  0.6512  0.7563
         MSE            0.8495  0.9216  0.6805  0.7309
168      MAE            0.9327  0.9657  0.6726  0.7650
         MSE            0.8988  0.9479  0.6982  0.7509

In [39]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            19155707.0  4376.3616  2706.0184  0.2176
         MSE            20330311.0  4508.7771  2985.8561  0.2242
96       MAE            39457330.0  6279.8901  3962.9127  0.3127
         MSE            36879912.0  6072.1143  4183.2212  0.3024
168      MAE            40141812.0  6335.6575  4107.5206  0.3157
         MSE            39366112.0  6272.8247  4301.4302  0.3125

# 2. Standard Scaler PatchTST

In [40]:
# Dynamic + default variables
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2  
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"
                
            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rat

In [4]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_default.csv'
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.4548  0.6744  0.4492  0.5338
              2         24        0.4541  0.6739  0.4488  0.5333
              1         96        0.7652  0.8747  0.6156  0.6938
              2         96        0.7669  0.8757  0.6170  0.6946
              1         168       0.8223  0.9068  0.6469  0.7179
              2         168       0.8208  0.9060  0.6468  0.7173
MAE           1         24        0.4620  0.6797  0.4367  0.5379
              2         24        0.4606  0.6787  0.4351  0.5371
              1         96        0.7871  0.8872  0.6019  0.7036
              2         96        0.7836  0.8852  0.6023  0.7021
              1         168       0.8368  0.9148  0.6298  0.7243
              2         168       0.8382  0.9155  0.6318  0.7248

In [5]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17643310.0  4200.3940  2682.8232  0.2089
              2         24        17569352.0  4191.5811  2676.9150  0.2084
              1         96        31436286.0  5606.8071  3670.9236  0.2792
              2         96        31604090.0  5621.7515  3682.2876  0.2800
              1         168       34401788.0  5865.3037  3863.6003  0.2923
              2         168       34455432.0  5869.8750  3867.4062  0.2925
MAE           1         24        17620972.0  4197.7344  2585.9958  0.2087
              2         24        17469412.0  4179.6426  2569.9910  0.2078
              1         96        32032500.0  5659.7261  3562.8826  0.2819
              2         96        32054014.0  5661.6265  3570.6907  0.2820
              1         168       34929708.0  5910.1362  3738.9783  0.2945
              2         168       35163840.0  5929.9106  3757.9302  0.2955

In [6]:
# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.4613  0.6792  0.4359  0.5375
         MSE            0.4545  0.6742  0.4490  0.5335
96       MAE            0.7854  0.8862  0.6021  0.7029
         MSE            0.7660  0.8752  0.6163  0.6942
168      MAE            0.8375  0.9152  0.6308  0.7246
         MSE            0.8215  0.9064  0.6469  0.7176

In [7]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            17545192.0  4188.6885  2577.9934  0.2083
         MSE            17606331.0  4195.9875  2679.8691  0.2086
96       MAE            32043257.0  5660.6763  3566.7866  0.2819
         MSE            31520188.0  5614.2793  3676.6056  0.2796
168      MAE            35046774.0  5920.0234  3748.4542  0.2950
         MSE            34428610.0  5867.5894  3865.5033  0.2924

In [8]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled')

# 3. MinMax Scaler Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [46]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [47]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Set the best learning rate based on pred_len
            if pred_len == "24":
                lr = 0.00001
            elif pred_len in ["96", "168"]:
                lr = 0.0001

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --dropout 0.1 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=True, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=3, learning_rate=1e-

	iters: 400, epoch: 1 | loss: 0.1093628
	speed: 0.0438s/iter; left time: 776.9736s
	iters: 500, epoch: 1 | loss: 0.1094514
	speed: 0.0412s/iter; left time: 726.8258s
	iters: 600, epoch: 1 | loss: 0.0901206
	speed: 0.0418s/iter; left time: 731.5526s
	iters: 700, epoch: 1 | loss: 0.1245983
	speed: 0.0414s/iter; left time: 721.5494s
	iters: 800, epoch: 1 | loss: 0.0997189
	speed: 0.0411s/iter; left time: 712.5943s
	iters: 900, epoch: 1 | loss: 0.0789374
	speed: 0.0418s/iter; left time: 720.4649s
-------------------------------------------------------------------------------------
Epoch: 1
Cost time: 00h:00m:39.69s
Steps: 906 | Train Loss: 0.1213056 Vali Loss: 0.1102202 Test Loss: 0.1235353
Validation loss decreased (inf --> 0.110220).  Saving model ...
Updating learning rate to 1e-05
	iters: 100, epoch: 2 | loss: 0.0458816
	speed: 0.1023s/iter; left time: 1751.3929s
	iters: 200, epoch: 2 | loss: 0.0474946
	speed: 0.0418s/iter; left time: 711.1581s
	iters: 300, epoch: 2 | loss: 0.0340314
	

In [48]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0224  0.1496  0.0999  0.5284
              2         24        0.0221  0.1487  0.0990  0.5251
              1         96        0.0416  0.2040  0.1469  0.7224
              2         96        0.0419  0.2047  0.1414  0.7251
              1         168       0.0419  0.2048  0.1461  0.7255
              2         168       0.0428  0.2069  0.1482  0.7330
MAE           1         24        0.0219  0.1481  0.0950  0.5229
              2         24        0.0224  0.1496  0.0968  0.5282
              1         96        0.0454  0.2131  0.1421  0.7545
              2         96        0.0441  0.2099  0.1398  0.7433
              1         168       0.0439  0.2096  0.1448  0.7427
              2         168       0.0448  0.2116  0.1448  0.7496

In [49]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        18418564.0  4291.6855  2771.5466  0.2134
              2         24        18167032.0  4262.2803  2737.8027  0.2119
              1         96        38027972.0  6166.6826  4192.3315  0.3071
              2         96        38741736.0  6224.2861  4020.8625  0.3100
              1         168       38667512.0  6218.3208  4159.8320  0.3098
              2         168       40122660.0  6334.2451  4268.5029  0.3156
MAE           1         24        17667154.0  4203.2314  2617.8315  0.2090
              2         24        18296418.0  4277.4312  2690.1646  0.2127
              1         96        40688736.0  6378.7725  4003.5076  0.3177
              2         96        38573712.0  6210.7739  3890.9275  0.3093
              1         168       39822152.0  6310.4795  4082.4695  0.3144
              2         168       41195184.0  6418.3477  4090.0542  0.3198

In [50]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled_minmax.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0221  0.1488  0.0959  0.5255
         MSE            0.0222  0.1492  0.0994  0.5267
96       MAE            0.0447  0.2115  0.1410  0.7489
         MSE            0.0418  0.2044  0.1441  0.7237
168      MAE            0.0444  0.2106  0.1448  0.7461
         MSE            0.0424  0.2059  0.1472  0.7293

In [51]:
# ALL 0.00001 - from 96 - BAD
# # 24 lr=0.000001; 96, 168 lr=0.00001 - BAD

inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            17981786.0  4240.3313  2653.9980  0.2108
         MSE            18292798.0  4276.9829  2754.6747  0.2127
96       MAE            39631224.0  6294.7732  3947.2175  0.3135
         MSE            38384854.0  6195.4844  4106.5970  0.3085
168      MAE            40508668.0  6364.4136  4086.2618  0.3171
         MSE            39395086.0  6276.2830  4214.1675  0.3127

# 4. MinMax Scaler PatchTST

In [52]:
log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [26]:
# Dynamic variables
losses = ["MSE", "MAE"]
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2 
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MAE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=True, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=

In [10]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_default.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0215  0.1467  0.0944  0.5180
              2         24        0.0215  0.1466  0.0945  0.5179
              1         96        0.0359  0.1895  0.1306  0.6709
              2         96        0.0359  0.1896  0.1308  0.6714
              1         168       0.0385  0.1962  0.1375  0.6951
              2         168       0.0384  0.1959  0.1374  0.6939
MAE           1         24        0.0220  0.1483  0.0922  0.5236
              2         24        0.0220  0.1483  0.0921  0.5236
              1         96        0.0370  0.1923  0.1281  0.6810
              2         96        0.0368  0.1917  0.1280  0.6789
              1         168       0.0392  0.1980  0.1343  0.7014
              2         168       0.0391  0.1976  0.1343  0.7001

In [11]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        17377448.0  4168.6265  2604.5605  0.2073
              2         24        17338256.0  4163.9233  2604.2974  0.2070
              1         96        31198698.0  5585.5796  3612.4434  0.2782
              2         96        31320716.0  5596.4912  3619.7280  0.2787
              1         168       34321564.0  5858.4609  3817.7300  0.2919
              2         168       34215236.0  5849.3789  3813.8789  0.2915
MAE           1         24        17364978.0  4167.1309  2519.1377  0.2072
              2         24        17280746.0  4157.0117  2512.1216  0.2067
              1         96        31669874.0  5627.5991  3512.3337  0.2803
              2         96        31654852.0  5626.2646  3514.6809  0.2802
              1         168       34749376.0  5894.8604  3702.5630  0.2937
              2         168       34745748.0  5894.5522  3707.1343  0.2937

In [12]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_0_1_relu.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0220  0.1483  0.0922  0.5236
         MSE            0.0215  0.1467  0.0945  0.5179
96       MAE            0.0369  0.1920  0.1280  0.6800
         MSE            0.0359  0.1895  0.1307  0.6712
168      MAE            0.0391  0.1978  0.1343  0.7007
         MSE            0.0384  0.1961  0.1374  0.6945

In [13]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            17322862.0  4162.0713  2515.6296  0.2069
         MSE            17357852.0  4166.2749  2604.4290  0.2072
96       MAE            31662363.0  5626.9319  3513.5073  0.2802
         MSE            31259707.0  5591.0354  3616.0857  0.2784
168      MAE            34747562.0  5894.7063  3704.8486  0.2937
         MSE            34268400.0  5853.9199  3815.8044  0.2917

In [14]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'minmax')

In [ ]:
# 24, 41 epoch: rmse:0.14708703756332397, mae:0.09022819995880127
# 24, 41 epoch: rmse:4097.39013671875, mae:2433.917236328125

# 24, 39 epoch: rmse:0.14707696437835693, mae:0.09025880694389343
# 24, 39 epoch: rmse:4091.692626953125, mae:2433.4794921875

# 96, 40 epoch: rmse:0.1888335943222046, mae:0.12493745982646942
# 96, 40 epoch: rmse:5546.99951171875, mae:3425.169189453125

# 96, 47 epoch: rmse:0.18904872238636017, mae:0.12519460916519165
# 96, 47 epoch: rmse:5797.2578125, mae:3607.472412109375

# 168, 31 epoch: rmse:0.19429442286491394, mae:0.13080841302871704
# 168, 31 epoch: rmse:5546.99951171875, mae:3425.169189453125

# 168, 32 epoch: rmse:0.1944178193807602, mae:0.13099423050880432
# 168, 32 epoch: rmse:5807.09423828125, mae:3613.380126953125